In [11]:
# Basic Libraries
import pandas as pd
import numpy as np


import yfinance as yf

# import plotly package for graphs
import plotly.graph_objs as go

# sklearn
from sklearn.preprocessing import MinMaxScaler

# Tensorflow
import keras
from keras.models import Sequential
from keras.layers import *

In [12]:
# getting data from Yahoo Finance
stock_name = 'NFLX'  # here you can change the name of stock ticker, for example we will take AMD ticker
data = yf.download(stock_name, start="2022-03-26", end="2023-03-29")
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-03-28,375.230011,380.279999,366.730011,378.510010,378.510010,4323400
2022-03-29,384.390015,396.500000,380.329987,391.820007,391.820007,5880700
2022-03-30,389.549988,392.700012,378.630005,381.470001,381.470001,4023300
2022-03-31,383.589996,384.000000,373.609985,374.589996,374.589996,4134700
2022-04-01,376.799988,380.869995,368.500000,373.470001,373.470001,4644200
...,...,...,...,...,...,...
2023-03-22,306.309998,306.450012,293.540009,293.899994,293.899994,5808000
2023-03-23,304.679993,322.779999,304.140015,320.369995,320.369995,15653300
2023-03-24,320.630005,331.829987,320.630005,328.390015,328.390015,12991700


In [13]:
# Choose only Close price of stock
data = data.filter(['Close'])
dataset = data.values

# Train data - 80%, test - 20%
training_data_len = int(np.ceil( len(dataset) * .80 ))

# Scale our data from 0 to 1
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

# Use our scaled data for training
train_data = scaled_data[0:int(training_data_len), :]
x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
    if i<= 61:
        print(x_train)
        print(y_train)
        print()

x_train, y_train = np.array(x_train), np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

[array([0.94096253, 1.        , 0.95409179, 0.92357503, 0.91860721,
       0.99858058, 0.9482368 , 0.89589709, 0.86839649, 0.84058549,
       0.80563316, 0.78833445, 0.81641157, 0.77516079, 0.76065638,
       0.80833879, 0.26533601, 0.22998449, 0.21800846, 0.19312489,
       0.1420714 , 0.09833665, 0.14703929, 0.10640942, 0.14677316,
       0.1485917 , 0.16695497, 0.09736088, 0.06475939, 0.02985146,
       0.05007766, 0.        , 0.03521846, 0.09434466, 0.08933244,
       0.10729652, 0.04799293, 0.07589266, 0.0886228 , 0.09345756,
       0.06196496, 0.09518743, 0.11102239, 0.12783325, 0.13781329,
       0.11772015, 0.17174539, 0.14464404, 0.1364826 , 0.1430029 ,
       0.16172102, 0.11709917, 0.07349748, 0.01472614, 0.00518961,
       0.0609448 , 0.03096035, 0.04054114, 0.02013754, 0.05553339])]
[0.06804174214279002]

[array([0.94096253, 1.        , 0.95409179, 0.92357503, 0.91860721,
       0.99858058, 0.9482368 , 0.89589709, 0.86839649, 0.84058549,
       0.80563316, 0.78833445, 0.81

In [14]:

# Build LSTM model
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape = (x_train.shape[1], 1)))
model.add(Dropout(0.35))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(25, activation = 'relu'))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Тrain the model
model.fit(x_train, y_train, batch_size=1, epochs=21)

# Structure of the model
keras.utils.plot_model(model, 'multi_input_and_output_model.png', show_shapes=True)

2023-07-20 13:25:08.568929: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-20 13:25:08.570280: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-20 13:25:08.571153: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/21


2023-07-20 13:25:08.902926: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-20 13:25:08.904404: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-20 13:25:08.905584: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

142/142 [==============================] - 4s 13ms/step - loss: 0.0201 - accuracy: 0.0000e+00
Epoch 2/21
142/142 [==============================] - 2s 13ms/step - loss: 0.0105 - accuracy: 0.0000e+00
Epoch 3/21
142/142 [==============================] - 2s 13ms/step - loss: 0.0105 - accuracy: 0.0000e+00
Epoch 4/21
142/142 [==============================] - 2s 13ms/step - loss: 0.0073 - accuracy: 0.0000e+00
Epoch 5/21
142/142 [==============================] - 2s 14ms/step - loss: 0.0067 - accuracy: 0.0000e+00
Epoch 6/21
142/142 [==============================] - 2s 13ms/step - loss: 0.0067 - accuracy: 0.0000e+00
Epoch 7/21
142/142 [==============================] - 2s 14ms/step - loss: 0.0061 - accuracy: 0.0000e+00
Epoch 8/21
142/142 [==============================] - 2s 13ms/step - loss: 0.0075 - accuracy: 0.0000e+00
Epoch 9/21
142/142 [==============================] - 2s 13ms/step - loss: 0.0057 - accuracy: 0.0000e+00
Epoch 10/21
142/142 [==============================] - 2s 13ms/ste

In [15]:
model.save('./LSTM.h5')
# Create test dataset
test_data = scaled_data[training_data_len - 60: , :]
x_test = []
y_test = dataset[training_data_len:, :]
for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])

x_test = np.array(x_test)

x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))
# Predict on test data
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

# For finding error we use RMSE formula, but MSE can be used too
rmse = np.sqrt(np.mean(((predictions - y_test) ** 2)))
print(f'RMSE LSTM: {rmse}')

2023-07-20 13:25:49.626547: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-20 13:25:49.627555: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-20 13:25:49.628307: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2/2 [==============================] - 0s 8ms/step
RMSE LSTM: 25.798510447561394


In [16]:

# Graphs
train = data[:training_data_len]
valid = data[training_data_len:]

#valid['Predictions'] = predictions
train_gr = np.reshape(train['Close'].values, (202,))
valid_gr = np.reshape(valid['Close'].values, (50,))

preds_gr = np.reshape(predictions, (50,))


In [17]:

x_train = list(range(0, len(train_data)))
x_valid = list(range(len(train_data)-1, len(dataset)))
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_train, y=train_gr, mode='lines+markers', marker=dict(size=4),  name='train', marker_color='#39304A'))
fig.add_trace(go.Scatter(x=x_valid, y=valid_gr, mode='lines+markers', marker=dict(size=4), name='valid', marker_color='#A98D75'))
fig.add_trace(go.Scatter(x=x_valid, y=preds_gr, mode='lines+markers', marker=dict(size=4), name='predictions', marker_color='#FFAA00'))
fig.update_layout(legend_orientation="h",
              legend=dict(x=.5, xanchor="center"),
              plot_bgcolor='#FFFFFF',
              xaxis=dict(gridcolor = 'lightgrey'),
              yaxis=dict(gridcolor = 'lightgrey'),
              title_text = f'{stock_name} LSTM data', title_x = 0.5,
              xaxis_title="Timestep",
              yaxis_title="Stock price",
              margin=dict(l=0, r=0, t=30, b=0))
fig.show()


In [18]:

# Predict stock prices for next month
data_new = yf.download(stock_name, start="2022-04-01", end="2023-07-30")

data_new = data_new.filter(['Close'])
dataset = data_new.values
training_data_len = len(dataset)

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

test_data = scaled_data[training_data_len - len(data_new): , :]
x_test = []
y_test = dataset[training_data_len:, :]
for i in range(20, len(test_data)):
    x_test.append(test_data[i-20:i, 0])

x_test = np.array(x_test)

x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))

hist_data_new = yf.download(stock_name, start="2021-04-01", end="2021-05-04")
hist_data_new = hist_data_new.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1)
hist_data_new = hist_data_new['Close']
hist_data_new = np.array(hist_data_new)
pred_lstm = model.predict(x_test)
pred_lstm = pred_lstm[:-1]
pred_lstm = scaler.inverse_transform(pred_lstm)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


2023-07-20 13:25:50.288489: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-20 13:25:50.289467: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-20 13:25:50.290165: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

10/10 [==============================] - 0s 4ms/step


In [19]:
# build graphs
preds_gr = np.reshape(pred_lstm, (304,))
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(0, 21)), y=hist_data_new, mode='lines+markers',  name='historical', marker_color='#39304A'))
fig.add_trace(go.Scatter(x=list(range(0, 21)), y=preds_gr, mode='lines+markers', name='predictions', marker_color='#FFAA00'))
fig.update_layout(legend_orientation="h",
              legend=dict(x=.5, xanchor="center"),
              plot_bgcolor='#FFFFFF',
              xaxis=dict(gridcolor = 'lightgrey'),
              yaxis=dict(gridcolor = 'lightgrey'),
              title_text = f'{stock_name} LSTM prediction', title_x = 0.5,
              xaxis_title="Timestep",
              yaxis_title="Stock price",
              margin=dict(l=0, r=0, t=30, b=0))
fig.show()
